In [1]:
import pickle
import pandas as pd

In [2]:
from scripts.flattening.flatten_day_batch_pkl import run_batch_flatten
from utils.preprocessing.clean_flattened_df import clean_flattened_dataframe
from utils.preprocessing.add_embedding_indices import add_embedding_indices
from utils.preprocessing.process_text_fields import process_text_fields, extract_race_phrases
from utils.io.embedding_io import save_embeddings_npz
#from utils.io.save_model_ready_df import save_model_ready_pickle

In [3]:
# Run flatten/merge module.
run_batch_flatten()

[INFO] 2025-03-28T04-59.json: 362 runners processed
        Valid horse IDs: 360/362
        class_num present: 277/362
[INFO] 2025-03-26T08-58.json: 296 runners processed
        Valid horse IDs: 295/296
        class_num present: 163/296
[INFO] 2025-03-30T04-57.json: 481 runners processed
        Valid horse IDs: 476/481
        class_num present: 228/481
[INFO] 2025-03-27T07-37.json: 387 runners processed
        Valid horse IDs: 383/387
        class_num present: 256/387
[INFO] 2025-03-31T04-51.json: 165 runners processed
        Valid horse IDs: 165/165
        class_num present: 165/165
[INFO] 2025-03-29T04-58.json: 659 runners processed
        Valid horse IDs: 653/659
        class_num present: 561/659
[SUCCESS] Flattened 2350 runners from 6 files
[✓] CSV saved to data/processed/2025-04-02T09-30.csv
[✓] Pickle saved to data/processed/2025-04-02T09-30.pkl


In [4]:
# Create a new datafrmae from the saved .pkl file.
new_df = pd.read_pickle('data/processed/2025-04-02T09-23.pkl')

In [5]:
# Analyse column attributes prior to cleaning
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   horse_id                 2350 non-null   object 
 1   horse_id_valid           2350 non-null   int64  
 2   course                   2350 non-null   object 
 3   country                  2350 non-null   object 
 4   going                    2350 non-null   object 
 5   GoingStick               0 non-null      object 
 6   distance_f               2350 non-null   float64
 7   field_size               2350 non-null   int64  
 8   race_class               2350 non-null   object 
 9   class_num                1650 non-null   float64
 10  class_label              1650 non-null   object 
 11  type                     2350 non-null   object 
 12  name                     2350 non-null   object 
 13  draw                     2158 non-null   float64
 14  age                     

In [6]:
df_clean = clean_flattened_dataframe(new_df)

✅ Filled NaNs in float columns: ['distance_f', 'class_num', 'draw', 'or', 'rpr', 'ts', 'trainer_ovr_runs', 'trainer_ovr_wins', 'trainer_ovr_win_pct', 'trainer_ovr_profit', 'trainer_last_14_runs', 'trainer_last_14_wins', 'trainer_last_14_win_pct', 'trainer_last_14_profit', 'jockey_ovr_runs', 'jockey_ovr_wins', 'jockey_ovr_win_pct', 'jockey_ovr_profit', 'jockey_last_14_runs', 'jockey_last_14_wins', 'jockey_last_14_win_pct', 'jockey_last_14_profit', 'rpr_rank', 'or_rank', 'rpr_zscore', 'or_zscore']


In [7]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 58 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   horse_id                 2350 non-null   object        
 1   horse_id_valid           2350 non-null   int64         
 2   course                   2350 non-null   object        
 3   country                  2350 non-null   category      
 4   going                    2350 non-null   category      
 5   distance_f               2350 non-null   float64       
 6   field_size               2350 non-null   int64         
 7   race_class               2350 non-null   category      
 8   class_num                2350 non-null   float64       
 9   class_label              1650 non-null   category      
 10  type                     2350 non-null   category      
 11  name                     2350 non-null   object        
 12  draw                     2350 non-

In [8]:
df_indexed, encoders = add_embedding_indices(df_clean)

In [9]:
# This will give you a preview of the new integer columns like:
# country_idx, going_idx, venue_idx, etc.
df_indexed.filter(like='_idx').head()

,country_idx,going_idx,sex_idx,type_idx,class_label_idx,headgear_idx,race_class_idx,venue_idx
0,4,6,2,1,1,12,0,7
1,4,6,2,1,1,0,0,7
2,4,6,2,1,1,17,0,7
3,4,6,2,1,1,0,0,7
4,4,6,1,1,1,4,0,7


In [10]:
df_indexed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 66 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   horse_id                 2350 non-null   object        
 1   horse_id_valid           2350 non-null   int64         
 2   course                   2350 non-null   object        
 3   country                  2350 non-null   category      
 4   going                    2350 non-null   category      
 5   distance_f               2350 non-null   float64       
 6   field_size               2350 non-null   int64         
 7   race_class               2350 non-null   category      
 8   class_num                2350 non-null   float64       
 9   class_label              1650 non-null   category      
 10  type                     2350 non-null   category      
 11  name                     2350 non-null   object        
 12  draw                     2350 non-

In [11]:
encoders

{'country': LabelEncoder(),
 'going': LabelEncoder(),
 'sex': LabelEncoder(),
 'type': LabelEncoder(),
 'class_label': LabelEncoder(),
 'headgear': LabelEncoder(),
 'race_class': LabelEncoder(),
 'venue': LabelEncoder()}

In [12]:
# Save encoders
import joblib
joblib.dump(encoders, 'data/processed/embedding_encoders_march-2025.pkl')

['data/processed/embedding_encoders_march-2025.pkl']

In [13]:
df_nlp, embeddings_dict, regex_features = process_text_fields(
    df_indexed,
    fields=["comment", "spotlight"],
    enable_regex=True
)

Embedding field: comment


Batches:   0%|          | 0/74 [00:00<?, ?it/s]

Embedding field: spotlight


Batches:   0%|          | 0/74 [00:00<?, ?it/s]

In [14]:
df_nlp.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 102 columns):
 #    Column                                    Dtype         
---   ------                                    -----         
 0    horse_id                                  object        
 1    horse_id_valid                            int64         
 2    course                                    object        
 3    country                                   category      
 4    going                                     category      
 5    distance_f                                float64       
 6    field_size                                int64         
 7    race_class                                category      
 8    class_num                                 float64       
 9    class_label                               category      
 10   type                                      category      
 11   name                                      object        
 12   draw

In [15]:
# Save the embeddings.
save_embeddings_npz(
    embeddings_dict,
    'data/processed/text_embeddings_march-2025.npz'
)

[✓] Saved 2 embedding blocks to data/processed/text_embeddings_march-2025.npz


In [30]:
# To load embeddings later.
from utils.embedding_io import load_embeddings_npz
# Optional; validate shape.
expected = {"comment": (1704, 384), "spotlight": (1704, 384)}
embeddings = load_embeddings_npz(
    "data/processed/text_embeddings_2025-03-31.npz",
    expected_schema=expected
)

[✓] Loaded 2 embedding blocks from data/processed/text_embeddings_2025-03-31.npz
[✓] Schema validation passed.


In [41]:
df_nlp.head(10)

,horse_id,horse_id_valid,course,country,going,distance_f,field_size,race_class,class_num,class_label,type,name,draw,age,sex,or,rpr,ts,lbs,headgear,last_run,form,trainer_id,trainer_ovr_runs,trainer_ovr_wins,trainer_ovr_win_pct,trainer_ovr_profit,trainer_last_14_runs,trainer_last_14_wins,trainer_last_14_win_pct,trainer_last_14_profit,jockey_id,jockey_ovr_runs,jockey_ovr_wins,jockey_ovr_win_pct,jockey_ovr_profit,jockey_last_14_runs,jockey_last_14_wins,jockey_last_14_win_pct,jockey_last_14_profit,...,class_label_idx,headgear_idx,race_class_idx,venue_idx,comment_vector,spotlight_vector,mentions_course_win_comment,mentions_course_win_spotlight,mentions_trip_change_comment,mentions_trip_change_spotlight,mentions_surface_comment,mentions_surface_spotlight,mentions_front_runner_comment,mentions_front_runner_spotlight,mentions_layoff_comment,mentions_layoff_spotlight,mentions_first_time_headgear_comment,mentions_first_time_headgear_spotlight,mentions_class_drop_comment,mentions_class_drop_spotlight,mentions_class_rise_comment,mentions_class_rise_spotlight,mentions_course_form_comment,mentions_course_form_spotlight,mentions_distance_form_comment,mentions_distance_form_spotlight,mentions_ground_form_comment,mentions_ground_form_spotlight,mentions_classy_rival_comment,mentions_classy_rival_spotlight,mentions_fitness_query_comment,mentions_fitness_query_spotlight,mentions_positive_trainer_note_comment,mentions_positive_trainer_note_spotlight,mentions_jockey_combo_comment,mentions_jockey_combo_spotlight,mentions_improver_flag_comment,mentions_improver_flag_spotlight,mentions_loser_flag_comment,mentions_loser_flag_spotlight
0,3679956,1,Dundalk (AW),IRE,Standard,7.0,11,,NaN,NaN,Flat,El Bello,3.0,6,gelding,68.0,71.0,35.0,142,p,21,020-88,21879,496.0,37.0,0.07,-210.37,12.0,0.0,0.00,-12.00,101060,135.0,21.0,0.16,-15.53,0.0,0.0,NaN,0.00,...,1,12,0,7,"[0.031336784, 0.033651523, 0.067663446, 0.0540...","[-0.03970722, 0.048932303, 0.06509631, 0.03706...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5148102,1,Dundalk (AW),IRE,Standard,7.0,11,,NaN,NaN,Flat,Roman Harry,6.0,5,gelding,64.0,68.0,43.0,142,,12,5326-0,21346,19.0,0.0,0.00,-19.00,4.0,0.0,0.00,-4.00,101012,505.0,43.0,0.09,-61.38,14.0,0.0,0.00,-14.00,...,1,0,0,7,"[0.033799525, 0.025958829, -0.021902591, 0.044...","[-0.0065255673, 0.059441153, -0.027289167, 0.0...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2455830,1,Dundalk (AW),IRE,Standard,7.0,11,,NaN,NaN,Flat,Skontonovski,11.0,8,gelding,70.0,74.0,47.0,142,v,114,38102-,10702,126.0,19.0,0.15,4.08,7.0,1.0,0.14,-3.00,103425,37.0,2.0,0.05,-13.00,5.0,0.0,0.00,-5.00,...,1,17,0,7,"[-0.092554845, -0.057024177, 0.027272813, -0.0...","[0.039343994, -0.016921623, -0.0012426724, -0....",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5301548,1,Dundalk (AW),IRE,Standard,7.0,11,,NaN,NaN,Flat,Solomon Coop,7.0,4,gelding,70.0,77.0,51.0,142,,28,760-14,20700,59.0,3.0,0.05,26.00,1.0,0.0,0.00,-1.00,101185,361.0,39.0,0.11,44.98,7.0,4.0,0.57,17.33,...,1,0,0,7,"[0.077566534, 0.0030810325, 0.0835577, -0.0036...","[0.07887819, -0.03196998, 0.05878525, 0.025985...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5021592,1,Dundalk (AW),IRE,Standard,7.0,11,,NaN,NaN,Flat,Nika Pika,2.0,4,filly,62.0,75.0,67.0,134,h,35,8-3809,17764,62.0,4.0,0.06,-11.50,3.0,0.0,0.00,-3.00,98736,143.0,12.0,0.08,-57.58,2.0,0.0,0.00,-2.00,...,1,4,0,7,"[-0.03139145, -0.019791834, 0.11644451, 0.0049...","[-0.024216084, 0.0015744533, 0.06750932, 0.027...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5329156,1,Dundalk (AW),IRE,Standard,7.0,11,,NaN,NaN,Flat,Signor Ferrari,9.0,4,gelding,71.0,79.0,59.0,134,,14,7-1167,27788,160.0,14.0,0.09,-63.31,4.0,0.0,0.00,-4.00,102115,138.0,10.0,0.07,-63.25,7.0,0.0,0.00,-7.00,...,1,0,0,7,"[0.012715498, -0.08430998, 0.030167041, 0.0104...","[-0.074624225, -0.023038115, 0.02531164, 0.083...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [16]:
df_nlp.to_pickle("data/processed/model_ready_march-2025.pkl")
df_nlp.to_csv("data/processed/model_ready_march-2025.csv", index=False)

In [54]:
# We have had issues with trainer/jockey stats not populating correctly.
trainer_cols = [col for col in df_nlp.columns if col.startswith("trainer_")]
jockey_cols = [col for col in df_nlp.columns if col.startswith("jockey_")]

df_nlp[trainer_cols + jockey_cols].isna().sum()


trainer_id                   0
trainer_ovr_runs            67
trainer_ovr_wins            67
trainer_ovr_win_pct        128
trainer_ovr_profit          67
trainer_last_14_runs        67
trainer_last_14_wins        67
trainer_last_14_win_pct    329
trainer_last_14_profit      67
jockey_id                    0
jockey_ovr_runs             67
jockey_ovr_wins             67
jockey_ovr_win_pct         128
jockey_ovr_profit           67
jockey_last_14_runs         67
jockey_last_14_wins         67
jockey_last_14_win_pct     224
jockey_last_14_profit       67
dtype: int64

In [55]:
# We to a sanity check to see if both these below return zero.
trainer_issues = df_nlp[
    (df_nlp['trainer_ovr_runs'] > 0) &
    (df_nlp['trainer_ovr_win_pct'].isna())
]
print(f"Trainer win % missing despite having runs: {len(trainer_issues)}")

jockey_issues = df_nlp[
    (df_nlp['jockey_ovr_runs'] > 0) &
    (df_nlp['jockey_ovr_win_pct'].isna())
]
print(f"Jockey win % missing despite having runs: {len(jockey_issues)}")


Trainer win % missing despite having runs: 0
Jockey win % missing despite having runs: 0
